In [1]:
import tensorflow as tf
import os


In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [3]:

data = tf.keras.utils.image_dataset_from_directory('data3')


In [4]:
data = data.map(lambda x,y: (x/255, y))


In [5]:


scaled_iterator = data.as_numpy_iterator()


batch = scaled_iterator.next() #images represented as numpy arrays

# print(batch[0].shape)

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [6]:
train_size = int(len(data)*.2) #to train
val_size = int(len(data)*.4) #to evaluate while training
test_size = int(len(data)*.4) #to test

In [7]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [9]:
model = Sequential()


In [10]:
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))




In [11]:
model.compile('adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])


In [12]:
model.summary()


In [14]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) #to save model


In [15]:
# hist = model.fit(train, epochs=3, validation_data=val,callbacks=[tensorboard_callback])
hist = model.fit(train, epochs=10, validation_data=val)


In [16]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [17]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [18]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy


In [19]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [20]:
for batch in test.as_numpy_iterator():
    X, y = batch
    predict_x=model.predict(X)
#     yhat = model.predict_classes(X)
    yhat=np.argmax(predict_x,axis=1)

    pre.update_state(y, yhat)
    re.update_state(y, yhat)
#     acc.update_state(y, yhat)

In [21]:
print(f'Precision:{pre.result().numpy()}, Recall: {re.result().numpy()},Accuracy: {acc.result().numpy()}')
